# Importar Librerías Requeridas
Importa las librerías necesarias, incluyendo pandas para la manipulación de datos y geopy para la geocodificación.

In [1]:
# Importar las librerías necesarias
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time
from datetime import datetime

# Cargar Datos desde Archivo Parquet
Carga el DataFrame desde el archivo Parquet especificado que contiene la columna 'dirección'.

In [2]:
# Cargar el archivo parquet
file_path = '../../data/processed/inmuebles24_departamentos_20251005.parquet'

# Leer el archivo Parquet en un DataFrame
df = pd.read_parquet(file_path)

In [3]:

# Verificar las primeras filas del DataFrame para asegurar que se cargó correctamente
df.head()

,precio_mxn,lote_m2,recamaras,baños,estacionamiento,es_amueblado,es_penthouse,cuenta_con_cocina_integral,cuenta_con_sala,cuenta_con_closet,...,cuenta_con_terraza,cuenta_con_comedor,cuenta_con_area_de_lavado,cuenta_con_salon_usos_multiples,cuenta_con_mantenimiento_incluido,cuenta_con_vigilancia_24_horas,direccion,colonia,cp,municipio
0,24000.0,100,2.0,2.0,1.0,0,0,1,1,0,...,0,1,0,0,0,0,san jeronimo lidice san jeronimo lidice la mag...,san jeronimo lidice,10200,la magdalena contreras
1,12000.0,50,1.0,1.0,1.0,1,0,0,1,1,...,0,0,1,0,1,0,callejon del prado barrio san francisco la mag...,el prado,9480,la magdalena contreras
2,34100.0,232,3.0,4.0,2.0,0,0,1,1,1,...,1,1,0,0,0,1,blvd. adolfo ruiz cortines 2775 san jeronimo l...,adolfo ruiz cortines,4630,la magdalena contreras
3,16000.0,165,2.0,1.0,1.0,0,0,0,1,0,...,0,1,0,0,0,0,magnolia 26 san jeronimo lidice la magdalena c...,san jeronimo lidice,10200,la magdalena contreras
4,26000.0,180,3.0,2.0,2.0,0,0,1,0,1,...,0,0,0,0,0,1,san marcos 11 pedregal 2 la magdalena contreras,san marcos,2020,la magdalena contreras


In [4]:
# Cargar el archivo parquet
file_path_prev = '../../data/processed/inmuebles24_departamentos_20250924.parquet'

# Leer el archivo Parquet en un DataFrame
df_prev = pd.read_parquet(file_path_prev)

In [5]:
df_prev.shape

(10233, 25)

In [6]:
df_prev[['direccion', 'colonia', 'cp', 'municipio']].value_counts()

direccion                                                                               colonia                 cp    municipio            
polanco polanco miguel hidalgo                                                          hidalgo                 1120  miguel hidalgo           76
campos eliseos polanco miguel hidalgo                                                   hidalgo                 1120  miguel hidalgo           59
miguel hidalgo lomas de sotelo lomas de sotelo miguel hidalgo                           hidalgo                 1120  miguel hidalgo           45
roma norte roma norte cuauhtemoc                                                        cuauhtemoc              6500  cuauhtemoc               41
alvaro obregon san pedro de los pinos san pedro de los pinos alvaro obregon             san pedro de los pinos  1180  alvaro obregon           35
                                                                                                                                  

In [7]:
df.shape

(13542, 25)

In [8]:
df[['direccion', 'colonia', 'cp', 'municipio']].value_counts()

direccion                                                                                           colonia     cp    municipio            
polanco polanco miguel hidalgo                                                                      hidalgo     1120  miguel hidalgo           94
miguel hidalgo lomas de sotelo lomas de sotelo miguel hidalgo                                       hidalgo     1120  miguel hidalgo           75
campos eliseos polanco miguel hidalgo                                                               hidalgo     1120  miguel hidalgo           66
roma norte roma norte cuauhtemoc                                                                    cuauhtemoc  6500  cuauhtemoc               58
miguel hidalgo anahuac anahuac i seccion miguel hidalgo                                             hidalgo     1120  miguel hidalgo           48
                                                                                                                                  

In [9]:
# Filtrar registros nuevos en df que no están en df_prev considerando 'direccion', 'colonia', 'cp', y 'municipio'
df_nuevos = df.merge(
    df_prev[['direccion', 'colonia', 'cp', 'municipio']],
    on=['direccion', 'colonia', 'cp', 'municipio'],
    how='left',
    indicator=True
).query("_merge == 'left_only'").drop(columns=['_merge'])

In [10]:

# Verificar las primeras filas del DataFrame resultante
df_nuevos.shape

(1413, 25)

In [11]:
df = df_nuevos.copy()

# Inicializar el Geocodificador Nominatim
Crea una instancia del geocodificador Nominatim, especificando un 'user_agent' único para tu aplicación.

In [12]:
# Inicializar el Geocodificador Nominatim
geolocator = Nominatim(user_agent="my_geocoding_app")  # Especificar un 'user_agent' único para la aplicación

# Definir Función para Obtener Coordenadas
Define una función que tome una dirección como entrada, use el geocodificador para encontrar su ubicación y devuelva la latitud y longitud. La función debe manejar casos en los que no se encuentre la dirección.

In [13]:
# Definir la función para obtener coordenadas
def obtener_coordenadas(direccion):
    """
    Toma una dirección como entrada y devuelve la latitud y longitud.
    Si no se encuentra la dirección, devuelve None para ambos valores.
    """
    try:
        # Usar el geocodificador para buscar la dirección
        #time.sleep(1)  # Añadir un retraso para respetar las políticas de uso del servicio
        ubicacion = geolocator.geocode(direccion, timeout=10)
        if ubicacion:
            return ubicacion.latitude, ubicacion.longitude
        else:
            return None, None
    except GeocoderTimedOut:
        # Manejar el caso en que el geocodificador exceda el tiempo de espera
        return None, None

# Aplicar Geocodificación a las Direcciones
Aplica la función de geocodificación a la columna 'dirección' del DataFrame para obtener las coordenadas de cada inmueble. Almacena los resultados en nuevas columnas llamadas 'latitud' y 'longitud'.

In [14]:
# Aplicar la función de geocodificación a la columna 'dirección'
df[['latitud', 'longitud']] = df['direccion'].apply(
    lambda direccion: pd.Series(obtener_coordenadas(direccion))
)

# Verificar las primeras filas del DataFrame para confirmar que se agregaron las columnas
df.head()

,precio_mxn,lote_m2,recamaras,baños,estacionamiento,es_amueblado,es_penthouse,cuenta_con_cocina_integral,cuenta_con_sala,cuenta_con_closet,...,cuenta_con_area_de_lavado,cuenta_con_salon_usos_multiples,cuenta_con_mantenimiento_incluido,cuenta_con_vigilancia_24_horas,direccion,colonia,cp,municipio,latitud,longitud
28676,14500.0,70,2.0,1.0,0.0,0,0,1,1,0,...,0,0,0,0,av. universidad altillo universidad coyoacan,Condominios para Empleados Federales,04340,coyoacan,19.337862,-99.183285
28756,23500.0,100,3.0,1.0,1.0,1,0,1,1,1,...,0,0,0,0,avenida universidad altillo universidad coyoacan,FOVISSSTE Oxtopulco,04310,coyoacan,19.337862,-99.183285
30476,6500.0,55,2.0,1.0,1.0,0,0,0,1,0,...,0,0,0,0,canal de tezontle dr. alfonso ortiz tirado izt...,Real del Moral,09040,iztapalapa,19.383617,-99.091983
30621,15000.0,48,2.0,1.0,1.0,0,0,0,1,0,...,0,0,0,1,avenida cuitlahuac 2989 azcapotzalco ciudad de...,Ampliación Cosmopolita,02680,azcapotzalco,19.468386,-99.176206
30672,12500.0,56,2.0,1.0,0.0,0,0,1,1,0,...,0,0,0,0,calle norte 87b recreo azcapotzalco,Un hogar para cada trabajador,02060,azcapotzalco,19.477841,-99.179759


# Verificar los Resultados
Muestra las primeras filas del DataFrame actualizado para verificar que las columnas 'latitud' y 'longitud' se han añadido y poblado correctamente.

In [15]:
# Muestra las primeras filas del DataFrame actualizado para verificar las columnas 'latitud' y 'longitud'
df[['direccion', 'latitud', 'longitud']].head()  # Mostrar solo las columnas relevantes para la verificación

,direccion,latitud,longitud
28676,av. universidad altillo universidad coyoacan,19.337862,-99.183285
28756,avenida universidad altillo universidad coyoacan,19.337862,-99.183285
30476,canal de tezontle dr. alfonso ortiz tirado izt...,19.383617,-99.091983
30621,avenida cuitlahuac 2989 azcapotzalco ciudad de...,19.468386,-99.176206
30672,calle norte 87b recreo azcapotzalco,19.477841,-99.179759


In [16]:
# Guardar el DataFrame con coordenadas en un nuevo archivo Parquet con la fecha
today_str = datetime.now().strftime('%Y-%m-%d')
output_path = f'../../data/processed/inmuebles24_departamentos_coordenadas_{today_str}.parquet'
df.to_parquet(output_path, index=False)
    

In [17]:
df.shape

(1413, 27)

In [18]:
df.latitud.isnull().sum(), df.longitud.isnull().sum()

(521, 521)

In [23]:
pd.set_option('display.max_colwidth', None)
df[df.latitud.isnull()][['direccion', 'municipio','latitud', 'longitud']]

,direccion,municipio,latitud,longitud
52809,privada nogal de castilla l7 pueblo nuevo alto la magdalena contreras,la magdalena contreras,NaN,NaN
52815,calle rolando garros 192 ampliacion aviacion civil venustiano carranza,venustiano carranza,NaN,NaN
52816,carlos marx 84 depto 3 ampliacion simon bolivar venustiano carranza,venustiano carranza,NaN,NaN
52821,carlos marx 84 depto 2 ampliacion simon bolivar venustiano carranza,venustiano carranza,NaN,NaN
52822,avenida galindo y villa 30 jardin balbuena venustiano carranza,venustiano carranza,NaN,NaN
...,...,...,...,...
66753,avenida de las granjas el jaguey azcapotzalco azcapotzalco ciudad de mexico,azcapotzalco,NaN,NaN
66757,centlapatl san martin xochinahuac azcapotzalco,azcapotzalco,NaN,NaN
66763,calzada lucio blanco 385 san juan tlihuaca azcapotzalco,azcapotzalco,NaN,NaN
66766,sabino 578 col. ampliacion del gas alcaldia azcapotzalco ciudad de mexico ampliacion del gas azcapotzalco,azcapotzalco,NaN,NaN


In [25]:
df[(df.latitud.isnull())&(df.municipio == 'miguel hidalgo')][['direccion', 'municipio','latitud', 'longitud']]

,direccion,municipio,latitud,longitud
54495,avenida de los campos eliseos 69 polanco v seccion polanco,miguel hidalgo,NaN,NaN
54596,lago andromaco 104 polanco i seccion polanco,miguel hidalgo,NaN,NaN
54603,polanco v seccion arquimedes miguel hidalgo cdmx polanco miguel hidalgo,miguel hidalgo,NaN,NaN
54710,neuchatel cuadrante polanco granada miguel hidalgo,miguel hidalgo,NaN,NaN
54721,be grand polanco anahuac i seccion miguel hidalgo,miguel hidalgo,NaN,NaN
...,...,...,...,...
64965,be grand alto polanco anahuac i seccion miguel hidalgo,miguel hidalgo,NaN,NaN
64968,felipe carrillo puerto san juanico miguel hidalgo,miguel hidalgo,NaN,NaN
64993,monte carpatos lomas de chapultepec miguel hidalgo,miguel hidalgo,NaN,NaN
64996,lope de vega 112 col. polanco v seccion alcaldia miguel hidalgo ciudad de mexico polanco v seccion polanco,miguel hidalgo,NaN,NaN


In [24]:
df[df.latitud.isnull()].municipio.value_counts()

municipio
miguel hidalgo            167
benito juarez              72
cuauhtemoc                 69
alvaro obregon             66
cuajimalpa de morelos      51
coyoacan                   19
tlalpan                    14
gustavo a. madero          13
azcapotzalco               13
iztapalapa                 11
xochimilco                  8
tlahuac                     7
venustiano carranza         6
iztacalco                   3
la magdalena contreras      2
Name: count, dtype: int64